In [1]:
# Connecting to the database file
import os
import glob
import csv
import numpy as np
import re
import sqlite3
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import numpy as np
import pickle

# cwd = os.getcwd()
# parentDir = os.path.dirname(cwd)
# sqlite_file = parentDir+"/mydatabase.db"
# conn = sqlite3.connect(sqlite_file)
# c = conn.cursor()
# command="CREATE TABLE IF NOT EXISTS Twitter(date TEXT, stockName TEXT, fav INTEGER, likes INTEGER, tweet TEXT, categories TEXT, UNIQUE(date, stockName) ON CONFLICT REPLACE);"
# c.execute(command)


/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#CREATE TABLE IF NOT EXISTS TwitterStockName(date TEXT, stockName TEXT, fav INTEGER, likes INTEGER, tweet TEXT, categories TEXT, sensitivity TEXT , UNIQUE(date, stockName) ON CONFLICT REPLACE);

In [3]:
import os
import glob
import csv
import numpy as np
import re
import sqlite3
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import numpy as np
import pickle
import codecs

cwd = os.getcwd()
parentDir = os.path.dirname(cwd)
sqlite_file = parentDir+"/mydatabase.db"
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
command="CREATE TABLE IF NOT EXISTS Twitter(date TEXT, stockName TEXT, fav INTEGER, likes INTEGER, tweet TEXT, categories TEXT, UNIQUE(date, stockName) ON CONFLICT REPLACE);"
c.execute(command)

def getTweetPath():
    global parentDir
    return (parentDir + "/Twitter")

def savePickleFile(d,varList):
    with open(d, 'wb') as f: 
        pickle.dump(varList, f)
    f.close()
def loadPickleFile(d):
    with open(d,'rb') as f:  # Python 3: open(..., 'rb')
        varList = pickle.load(f)
    f.close()
    return varList 

def makeDirectory(folder_name1,sub_folder):
    directory = os.path.join(folder_name1 , sub_folder)
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

def getOutputDir(folderName,fileName):
    global parentDir
    outputPath = makeDirectory(parentDir, folderName) 
    d = outputPath + fileName
    return d


#training Classification

def trainCategoriesClassifier():
    twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

    text_clf = Pipeline([('vect', CountVectorizer()),('tfidf',TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),])
    #text_clf = Pipeline([('vect', CountVectorizer()),('tfidf',TfidfTransformer()),('clf', MultinomialNB()),])

    #Bayes Naive Train Model
    text_clf.fit(twenty_train.data, twenty_train.target)
    savePickleFile(getOutputDir('pickleObj','/classificationObj.pkl'),[text_clf])
#     with open('./pickleObj/CatClassifier.pkl', 'wb') as f: 
#         pickle.dump([text_clf], f)
#     f.close()
    
# def loadCategoriesClassifier():
#     with open('./pickleObj/CatClassifier.pkl','rb') as f:  # Python 3: open(..., 'rb')
#         text_clf = pickle.load(f)
#     f.close()
#     return text_clf

def getIntoDir():
    os.chdir(getTweetPath())
    
def getOutDir():
    path = '..'
    os.chdir(path)
             
def loadTweetsFile():
    getIntoDir()
    extension = 'csv'
    result = [i for i in glob.glob('*.{}'.format(extension))]
    getOutDir()
    return result

def removeSubString(s,sub):
    start = s.find(sub)
    end = s.find(' ', start)
    if (end ==-1):
        end = len(s)
    tmp = s[:start] + s[end:]
    if (tmp.find(sub)!=-1):
        tmp = removeSubString(tmp,sub)
    return (tmp)

def readFile(fname):
    getIntoDir()
    ans = []
    with open(fname) as f:
        reader = csv.reader(f,delimiter=';')
        for row in reader:
            tmp = [row[0],row[1],row[2],row[3],row[4]]
            ans.append(tmp)
    getOutDir()
    return ans

def cleanString(s):
    tmp = removeSubString(s,"http")
    tmp = removeSubString(tmp,"@")
    tmp = removeSubString(tmp,"#")
    tmp = removeSubString(tmp,"&amp")
    tmp = re.sub('[^a-zA-Z ]+', ' ', tmp)
    tmp = re.sub( '\s+', ' ', tmp ).strip()
    return tmp.lower()

def getCategory(result):
    text_clf=loadPickleFile(getOutputDir('pickleObj','/classificationObj.pkl'))[0]
#     print (text_clf)
    ans = []
    for f in result:
        ans = ans+readFile(f)
    data = np.array(ans)
    #remove files
    for f in result:
        if (os.path.isfile(f)):
            os.remove(f)
    
    tweetL = data[:,4].tolist()
    tweetLClean=[]
    for i in tweetL:
        tmp = cleanString(i)
        tweetLClean.append(tmp)
    return data,text_clf.predict(tweetLClean)

def getData():
    twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
    data,predicted = getCategory(loadTweetsFile())
    cat=twenty_train.target_names
    tmp=[]
    for i in predicted:
        tmp.append(cat[i])
    predicted = tmp
    predictedNP=np.transpose(np.array(predicted))
    return data,predictedNP
    
if (not (os.path.isfile(getOutputDir('pickleObj','/classificationObj.pkl')))):
    trainCategoriesClassifier()

finalData,finalPredicted = getData()



MemoryError: 

In [ ]:
def insertToDatabase(data,predicted,c):
    #c is connector SQL
    table_name = "Twitter"
    for i in range(len(data)):
        values = (data[i][0],data[i][1],data[i][2],data[i][3],data[i][4],predicted[i])
        c.execute("INSERT INTO {tn} VALUES(?,?,?,?,?,?)".format(tn=table_name),values)
                  
# In[10]:
insertToDatabase(finalData,finalPredicted,c)

In [ ]:
conn.commit()

conn.close()